<a href="https://colab.research.google.com/github/karthikyandrapu/Blackbucks_IIDT_AI-ML-DS_Internship/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Credit Card Fraud Detection**

Data Preprocessing

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ML/creditcard.csv')

# Perform EDA (Exploratory Data Analysis)
print(df.head())
print(df.describe())

# Normalize or standardize the data
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))

Feature Engineering

In [ ]:
# Feature selection
X = df.drop(['Time', 'Class'], axis=1)
y = df['Class']

# Dimensionality reduction
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

Model Selection

In [ ]:
# Choose appropriate machine learning algorithms
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

Model Training

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train the selected model
model.fit(X_train, y_train)

# Tune hyperparameters (GridSearchCV)
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

Model Evaluation

In [ ]:
# Evaluate model performance using appropriate metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score

# Evaluation on test set
y_pred = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('ROC AUC Score:', roc_auc_score(y_test, y_pred))

# Cross-validation for robustness check
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='roc_auc')
print('Cross-validation ROC AUC:', cv_scores.mean())

Model Deployment

In [ ]:
# Save the trained model
import joblib

joblib.dump(best_model, 'credit_card_fraud_detection_model.pkl')

# Deploy the model using a web framework (Flask)
from flask import Flask, jsonify, request
import joblib

app = Flask(__name__)
model = joblib.load('credit_card_fraud_detection_model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = [data['feature1'], data['feature2'], ...]
    prediction = model.predict([features])
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)